In [72]:
##Requisição junto ao servidor do IBGE.

import requests
import pandas as pd

headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

municipios_jsons = requests.get('https://servicodados.ibge.gov.br/api/v2/malhas/16/?formato=application/vnd.geo+json&resolucao=5', headers = headers)
municipios_json = municipios_jsons.json()
from pandas.io.json import json_normalize
data_mun = municipios_json['features']
df_mun = json_normalize(data_mun)
df_mun

<ipython-input-72-53317ecf8253>:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_mun = json_normalize(data_mun)


,type,properties.codarea,properties.centroide,geometry.type,geometry.coordinates
0,Feature,1600055,"[-52.26801275605981, 1.652173220506485]",MultiPolygon,"[[[[-52.0158, 2.4365], [-52.041, 2.4237], [-52..."
1,Feature,1600154,"[-52.51908621590008, 1.1573395871948589]",MultiPolygon,"[[[[-52.8679, 2.0247], [-52.8119, 1.9804], [-5..."
2,Feature,1600279,"[-53.31060613663834, 1.1016546223258232]",MultiPolygon,"[[[[-53.2791, 2.1859], [-53.2469, 2.151], [-53..."
3,Feature,1600808,"[-52.06116725155589, -0.9962266780545679]",MultiPolygon,"[[[[-51.7783, -1.1404], [-51.809, -1.1571], [-..."
4,Feature,1600212,"[-50.48679166107683, 1.0352149941487467]",MultiPolygon,"[[[[-50.0672, 1.2167], [-50.069, 1.2022], [-50..."
5,Feature,1600253,"[-50.64007883268975, 0.5381767024973914]",MultiPolygon,"[[[[-50.4098, 0.6233], [-50.4365, 0.6001], [-5..."
6,Feature,1600105,"[-50.581515594256835, 1.7652460029401984]",MultiPolygon,"[[[[-50.1812, 1.9009], [-50.1612, 1.8992], [-5..."
7,Feature,1600204,"[-51.40413420306525, 2.4312781287715963]",MultiPolygon,"[[[[-50.7025, 2.1382], [-50.7287, 2.1476], [-5..."
8,Feature,1600238,"[-51.416859513418466, 0.9571983806885738]",MultiPolygon,"[[[[-50.893, 0.9933], [-50.9072, 0.9627], [-50..."
9,Feature,1600303,"[-50.701772540974595, 0.5543179259576944]",MultiPolygon,"[[[[-50.0939, 0.702], [-50.1569, 0.7051], [-50..."


In [55]:
##Criação do Mapa
import folium
mapa = folium.Map(location =[-22.7864889, -50.6786708], zoom_start=7, tiles='Stamen Terrain')
folium.GeoJson(municipios_json).add_to(mapa)
mapa

In [15]:
##Dados sobre a vacinação COVID-19
import pandas as pd
import io
import numpy as np

In [95]:
results_caract = []
results_series = []
seq_url = ['AP']
for i in seq_url:
    i = str(i)
    url = 'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-04-19/uf%3D'+ i + '/part-00000-51e5ff2c-513f-4d37-8a45-5ee2c98cb342.c000.csv'
    print('Estado sendo extraído: '+ i)
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep = ';')
    c['UF'] = i    
    c['indicador'] = 1    
    caract = c.groupby(['paciente_endereco_coIbgeMunicipio','paciente_endereco_nmMunicipio', 'vacina_descricao_dose']).agg({'indicador': [np.sum]}).reset_index()
    results_caract.append(caract)
results_caract = pd.concat(results_caract)
df1dose = results_caract.loc[results_caract['vacina_descricao_dose'] =='\xa0\xa0\xa0\xa01ª\xa0Dose']
df2dose = results_caract.loc[results_caract['vacina_descricao_dose'] =='\xa0\xa0\xa0\xa02ª\xa0Dose']

Estado sendo extraído: AP


In [97]:
results_caract['paciente_endereco_coIbgeMunicipio'] = results_caract['paciente_endereco_coIbgeMunicipio'].astype(int)
results_caract['paciente_endereco_coIbgeMunicipio'] = results_caract['paciente_endereco_coIbgeMunicipio'].astype('object')

,paciente_endereco_coIbgeMunicipio,paciente_endereco_nmMunicipio,vacina_descricao_dose,indicador
,,,,sum
0,110002.0,ARIQUEMES,1ª Dose,3
2,110004.0,CACOAL,1ª Dose,5
4,110005.0,CEREJEIRAS,1ª Dose,1
6,110008.0,COSTA MARQUES,1ª Dose,1
7,110012.0,JI-PARANA,1ª Dose,7
...,...,...,...,...
1129,522140.0,TRINDADE,1ª Dose,3
1131,522145.0,TROMBAS,1ª Dose,1
1132,522185.0,VALPARAISO DE GOIAS,1ª Dose,2


In [14]:
url_link = requests.get('https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2020/variaveis/9324?localidades=N6[all]')
url_data = url_link.json()

In [93]:
from pandas.io.json import json_normalize
data_rec = url_data[0]['resultados'][0]['series']
df = json_normalize(data_rec)
df

<ipython-input-93-6d96dfa7ebb1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data_rec)


,localidade.id,localidade.nivel.id,localidade.nivel.nome,localidade.nome,serie.2020
0,1100015,N6,Município,Alta Floresta D'Oeste - RO,22728
1,1100023,N6,Município,Ariquemes - RO,109523
2,1100031,N6,Município,Cabixi - RO,5188
3,1100049,N6,Município,Cacoal - RO,85893
4,1100056,N6,Município,Cerejeiras - RO,16204
...,...,...,...,...,...
5565,5222005,N6,Município,Vianópolis - GO,13977
5566,5222054,N6,Município,Vicentinópolis - GO,8873
5567,5222203,N6,Município,Vila Boa - GO,6312
5568,5222302,N6,Município,Vila Propício - GO,5882
